**General description of this notebook.**

There exists a python package for vortex dynamics named VortexFitting:

    - its reference is https://doi.org/10.1016/j.softx.2020.100604 (G. Lindner, Y. Devaux, S. Miskovic, 
      "VortexFitting: a post-processing fluid mechanics tool for vortex identification)

    - its GitHub page is https://github.com/guilindner/VortexFitting

    - its documentations page is https://guilindner.github.io/VortexFitting/
    
VortexFitting package has been integrated into PIVPY.
This tutorial shows how to work with VortexFitting package through PIVPY.

**How the PIVPY to VortexFitting conversion is implemented.**

Per Dr. Liberzon suggestion, VortexFitting has its own class `VelocityField` for storing velocity fields. One way to create a variable of this class is to read the field from a NetCDF file. Therefore, the function `pivpyTovf()` from the inter.py file of PIVPY package, takes velocity field in the form of PIVPY class object, saves this field to a NetCDF file (which is stored on the user's computer, user is let to decide what to do with this file afterwords) and reads the NetCDF file into VortexFitting class object. From there, any fucntion from VortexFitting package will be able to work with it.

**Peculiarities of VortexFitting.**

VortexFitting is a Python package, it is not a Python library (i.e., you just run it from console as a 
piece of software - you do not use it like, for instance, `import numpy` in your code). Here is the work around
describing how to use VortexFitting as a library.

Either clone VortexFitting to your computer or go to its GitHub page. Navigate to the folder vortexfitting. There you will see a lot of .py files. Those files contain classes and functions of
VortexFitting. You can open each of those files, read the descriptions of the functions and see
what they can do. Once you have determined what functions (or class) you want to use in your code you
can use `import` command, but in a specific manner.

For example, I want to plot vorticity fields. I found that the file fitting.py contains the function `plot_fields()` that can plot vorticity. Then in my code, I type the following:
```
import vortexfitting.fitting as vfFitting
vfFitting.plot_fields(<the name of my field>, vorticity)
```

**Here is a custom made example with a velocity field that is also used to test the function pivpyToVf().**

In [ ]:
import pathlib
import importlib.resources
import numpy as np
import matplotlib.pyplot as plt
import vortexfitting.schemes as vfSchemes
import vortexfitting.fitting as vfFitting
import vortexfitting.detection as vfDetection
import vortexfitting.output as vfOutput
from netCDF4 import Dataset
from pivpy import io, graphics, inter 

In [ ]:
ls /home/user/Downloads/GuyRon_Analysis/

In [ ]:
# For Python 3.9+
path = '/home/user/Downloads/GuyRon_Analysis/'
# Convert to pathlib.Path if not already
path = pathlib.Path(path)
assert path.exists()

In [ ]:
# read VEC files into PIVPy structure (xarray)
d = io.load_directory(path) # maybe a bit long, see also how to read a single VEC file
# https://github.com/alexlib/pivpy/blob/master/examples/notebooks/test_reading_different_files.ipynb

In [ ]:
d.head()

In [ ]:
d.to_netcdf(path / 'data.nc')

In [ ]:
# Create an OpenPIV .txt file with velocity field (interTest.txt), put it into a folder and specify the path to it:
# openpivTxtTestFile = path / "openpiv_txt" / "interTest.txt" 
# Think up a name for the future NetCDF file (which is created as an itermediate file as described above),
# think where it will be located and create such a path. To be totally clear: the NetCDF file
# testInterCreates_nc.nc does not exist yet. We have just created a name for it.
saveToDir = path / "interTest"
if not saveToDir.exists():
    saveToDir.mkdir()
    
saveNcFile  = path / "interTest" / "testInterCreates_nc.nc"
# File with vortices created by VortexFitting:
saveVortexFile = path / "interTest" / "vortices.dat"
# Directory to save the plots to:


In [ ]:
from pivpy import pivpy 
d.isel(t=0).piv.quiver()

In [ ]:
# test on 5 first maps:
vfField = inter.pivpyTOvf(d.isel(t=slice(0,4)), saveNcFile)

# OR test on all maps:

# Convert the PIVPY object to the VortexFitting object using the function pivpyTOvf() from inter.py module.
# vfField = inter.pivpyTOvf(d, saveNcFile)

In [ ]:
# As mentioned before, conversion of the PIVPY object to the VortexFitting object yields an auxiliary 
# NetCDF4 file. It is left to the user to decide what to do with the file. Just in case, this is a
# brief example of how to work with a NetCDF4 file.
# First, read the file:
nc = Dataset(str(saveNcFile))
# Second, access its data:
print("nc['grid_n']:\n{}".format(nc['grid_n'][:])) # the name of x coordinate in the VortexFitting class
print("\nnc['grid_z']:\n{}".format(nc['grid_z'][:])) # the name of y coordinate in the VortexFitting class
print("\nnc['velocity_n']:\n{}".format(nc['velocity_n'][:])) # the name of u velocity component in the VortexFitting class
print("\nnc['velocity_s']:\n{}".format(nc['velocity_s'][:])) # the name of v velocity component in the VortexFitting class
print("\nx coordinate nc-file shape: {}".format(nc['grid_n'].shape))
print("y coordinate nc-file shape: {}".format(nc['grid_z'].shape))
print("u velocity component nc-file shape: {}".format(nc['velocity_n'].shape))
print("v velocity component nc-file shape: {}".format(nc['velocity_s'].shape))


Having obtained the VortexFitting object, you can apply any VortexFitting functions to it. Here are some examples.
**IMPORTANT: see the module \_\_main.py\_\_ in VortexFitting package for the exhaustive list of all the ways
you can work with VortexFitting**.

**Example 1**: obtaining the fields of u, v, x and y.

In [ ]:
# IMPORTANT: the object of VortexFitting class does not store u, v, x and y as they are.
# There, u and v are, essentially, the fluctuating components of u and v velocity components (i.e,
# VortexFitting automatically subtracts the means from u and v); y coordinate is automatically 
# inversed by VortexFitting. For more information see the source code for class VelosityField (lines
# 87-108 under the "if file_type == 'piv_netcdf':" statement) in the classes.py module of 
# VortexFitting package.
print("vfField.u_velocity_matrix:\n{}".format(vfField.u_velocity_matrix[:]))
print("\nvfField.v_velocity_matrix:\n{}".format(vfField.v_velocity_matrix[:]))
print("\nvfField.x_coordinate_matrix:\n{}".format(vfField.x_coordinate_matrix[:]))
print("\nvfField.y_coordinate_matrix:\n{}".format(vfField.y_coordinate_matrix[:]))

In [ ]:
# Compare them to the original velocity and coordinate fields.
print("d['u'].values:\n{}".format(d['u'].values))
print("\nd['v'].values:\n{}".format(d['v'].values))
print("\nd.coords['x'].values:\n{}".format(d.coords['x'].values))
print("\nd.coords['y'].values:\n{}".format(d.coords['y'].values))

**Example 2**: ploting colored contours of velocity and vorticity fields.

In [ ]:
# To plot vorticity field, vorticity must be calculated first.
# VortexFitting offers three methods to approximate the derivatives going into the definition of
# vorticity (see the functions in the schemes.py module of VortexFitting package). They are 
# second_order_diff(vfield), least_square_diff(vfield) and fourth_order_diff(vfield). They way they
# can be used is exactly the same. Let me examplify their usage on fourth_order_diff(vfield) function.
# IMPORTANT to remember that this vorticity is based on the fluctuating velocity components.
vfFieldDerivative = vfSchemes.fourth_order_diff(vfField)
vorticity = vfFieldDerivative['dvdx'] - vfFieldDerivative['dudy']
print("Vorticity={}".format(vorticity))

In [ ]:
# The function plot_fields() from the module fitting.py of VortexFitting package offers a way to
# plot colored contours of vorticity.
# Run the cell and not the following important features of this function.
# 1) It plots not only colored contours of vorticity, but also colored contours of u,v and w fluctuating
# components of velocity. When the object of VortexFitting class is being created, VortexFiting demands
# w component. It wouldn't allow the absence of w component. w component may be None, but it must 
# exist. That's why the w plot is all white.
# 2) Even though the function plot_fields() accepts any "detection field" - not just vorticity - it
# will always call the "detection field" plot "vorticity".
# 3) Under the hood, the function plot_fields() makes use of the function plt.imshow() from matplotlib.
# The function plt.imshow() converts our fields to an image. Our fields - for instance, u field - 
# have each data point assigned to the center of the interrogation window. The size of the interrogation
# window might be, say, 16pix (or 32pix or 64pix). But the function plt.imshow() - when converting the
# field to the image - treats every interrogation window as a pixel. That is why the vertical and
# horizontal spans of the collored contours created by plot_fields() are smaller than the corresponding
# spans of our original fields.
# The flow setup I am using for this example has a rather peculiar upward flow along an inclined wall.
vfFitting.plot_fields(vfField, vorticity)

**Example 3**: ploting colored contour of vorticity field imposed on the field of the fluctuating velocity components.

In [ ]:
# The function plot_quiver() fro the fitting.py module of VortexFitting package offers a way to impose
# the colored contour of vorticity field on the plot of the fluctuating velocity components. Here is
# an example of how to use it.
# The flow setup I am using for this example has a rather peculiar upward flow along an inclined wall.
x2D, y2D = np.meshgrid(vfField.x_coordinate_matrix, vfField.y_coordinate_matrix)
vfFitting.plot_quiver(x2D, y2D, vfField.u_velocity_matrix, vfField.v_velocity_matrix, vorticity)

In [ ]:
# To make sure VortexFitting plots the right field, let's do two tests.
# First, let's use matplotlib package to plot the field. The idea is that if another package results
# in the identical field, VortexFitting is validated.
plt.contourf(vorticity, extent=[x2D[0][0], x2D[0][-1], y2D[0][0], y2D[-1][0]])
plt.quiver(x2D, y2D, vfField.u_velocity_matrix, vfField.v_velocity_matrix)
plt.show()

In [ ]:
# Second, let's use matplotlib package to plot the original fields (the ones given by PIVPY object).
# The idea is the same - use another object to get the identical plot. Note, however, that the PIVPY
# object - as oppose to the VortexFitting object - stores the full velocity components, not just
# thir fluctuating parts.
x = d.coords['x'].values.flatten()
y = d.coords['y'].values.flatten()
X, Y = np.meshgrid(x, y)
Uvel = np.nan_to_num(d['u'].isel(t=0).values, copy=False); Uvel = np.subtract(Uvel,Uvel.mean())
Vvel = np.nan_to_num(d['v'].isel(t=0).values, copy=False); Vvel = np.subtract(Vvel,Vvel.mean())
plt.quiver(X, Y, Uvel.T, Vvel.T) 
plt.show()


In [ ]:
# And the same thing can be done using PIVPY native syntax:
graphics.quiver(d, arrScale=10) 
plt.show()

**Example 3**: vortex detection.

In [ ]:
# VortexFitting package offers 3 vortex detection methods (for the theoretical basis, see their 
# documentation at https://guilindner.github.io/VortexFitting/methodology.html). They are: Q criterion,
# Delta criterion and Swirling strength criterion. They are represented by the corresponding functions
# calc_q_criterion(vfield), calc_delta_criterion(vfield) and calc_swirling(vfield) located in the 
# module detection.py of VortexFitting package. Their usage is exactly the same. I'm going to examplify
# the use of the Q criterion.
vortexDetect = vfDetection.calc_q_criterion(vfField) # detect the vortices; to play with other methods
                                                     # just substitute "calc_q_criterion" with either
                                                     # "calc_swirling" or "calc_delta_criterion" and run the cell
sst = 0.1 # swirling strength detection threshold
bs = 3 # box size (to avoid vortices overlapping) - integer distance between two vortices in mesh units
peaks = vfFitting.find_peaks(vortexDetect, sst, bs) # detect the peaks of the swirls
print('Vortices found: ', len(peaks[0]))
vortices_counterclockwise, vortices_clockwise = vfFitting.direction_rotation(vorticity, peaks) # detect the direction of rotation
rmax = 100 # initial guess of the vortex radius
ch = 0.75 # correlation threshold - a parameter used in get_vortices() function; reduce it if the vortex is too big
vortices = vfFitting.get_vortices(vfField, peaks, vorticity, rmax, ch) # pick only the real vorticies (remove the noise)
print('---- Accepted vortices ----')
print(len(vortices))
# Plot the the detected vorticies imposed on the fielf of the fluctuating velocities.
x_index, y_index, u_data, v_data = vfFitting.window(vfField, 0, 0, rmax)
vfFitting.plot_quiver(x_index, y_index, u_data, v_data, vortexDetect)
# Plot location and rotation of the vortices.
# The function plot_detect() uses plt.imshow() under the hood, which, again, means that
# every one of our interrogation windows is presented as a pixel.
vfFitting.plot_detect(vortices_counterclockwise, vortices_clockwise, vortexDetect, 0) # 0 flag means don't invert X and Y axes for plotting; 1 flag does the opposite
# Plot and create an outpot file with identified vortices.
# Note, that the function create() from the module output.py of VortexFitting package 
# cannot be used because it requires arguments from the console. Instead, just implement this 
# function here:
with saveVortexFile.open("w") as outfile:
    outfile.write("TITLE=\"Vortex characteristics evolution\"\n")
    outfile.write("Variables=\"time\",\"radius\",\"gamma\",\"xcenter\",\"ycenter\","
                  "\"u_advection\",\"v_advection\",\"correlation\",\"vtheta\"\n")
    outfile.write("DATASETAUXDATA Detection_method=\"{}\"\n".format("Q criterion"))
    outfile.write("DATASETAUXDATA Scheme=\"{}\"\n".format('fourth order')) # see the cell where we calculated vorticity
    outfile.write("DATASETAUXDATA Box_size=\"{}\"\n".format(bs))
    outfile.write("DATASETAUXDATA Detection_threshold=\"{}\"\n".format(sst))
    outfile.write("DATASETAUXDATA Rmax=\"{}\"\n".format(rmax))
    outfile.write("DATASETAUXDATA Correlation_threshold=\"{}\"\n".format(ch))
    outfile.write("DATASETAUXDATA Mean_file=\"{}\"\n".format("mean field subtracted"))
    outfile.write("DATASETAUXDATA File_type=\"{}\"\n".format('piv_netcdf'))
    outfile.write("ZONE T=\"0\", SOLUTIONTIME=0\n")
# Plot accepted vortices.
# The function plot_accepted() yields two plots: accepted vortices as contours and accpeted vortices
# as plt.imshow(). Pay attention, that plt.imshow() treats every interrogation window as 1 pixel.
vfFitting.plot_accepted(vfField, vortices, vortexDetect, str(saveToDir), 0, 'png') # the argument 0 refers to the time step
# Add vorticies information to the output file:
vfOutput.write(vortices, str(saveToDir), 0) # the argument 0 refers to the time step
# Let's see if the files were created:
print("All the files in the saving directory saveToDir:")
for path in saveToDir.iterdir(): print(path.name) # You will see the files accepted_0.png and 
                                                  # accepted_0.svg which are the plots of the accepted vortices;
                                                  # files linked_0.svg and meshed_0.png which are the plots of
                                                  # the detected vortices; the file vortices.dat contains
                                                  # vortices data (i.e., it's an analog of OpenPIV .txt file)